In [1]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

In [12]:
dataset = 'mnist'
model_name = dataset + '_pruned_sparcity-90'
pruned_keras_file = './models/' + dataset + '/' + model_name + '.h5'
tflite_model_file = './models/' + dataset + '/' + model_name + '.tflite'

In [13]:
#if pruned model
with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(pruned_keras_file)
#if non-pruned model
#final_model = tf.keras.models.load_model(pruned_keras_file)

In [14]:
#if pruned model
final_model = sparsity.strip_pruning(restored_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [15]:
final_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Froze 12 variables.
INFO:tensorflow:Converted 12 variables to const ops.
